# Extracting and querying facts with GPT-3: Study 
In this study, we will examine how to use GPT-3 to extract facts from Natural Language (NL) utterances and, later, query those facts from a database. To do so, we will:
  - Define some simple NL example utterances.
  - Prompts to extract facts from those utterances.
  - Auxiliary mechanisms to store the extracted facts. Here, we will use a simple Pandas Dataframe.
  - Prompts to help querying the facts from the database. As we shall see, even here we can leverage GPT-3 to help us.
  - Auxiliary mechanisms to apply all of the above to the example utterances, in convenient way for our study.

Please not that in order to be able to execute this notebook, you need to have a working [OpenAI API](https://openai.com/api/) key. 

## Setup

In [2]:
import openai
import os
import pandas as pd

**For security**, it is recommended to input this key in a way that is not recorded in the notebook. Rather, define it as an environment variable called `OPENAI_API_KEY`. In Windows, go to *System Properties* -> *Environment Variables...* and add a new one. You will probably have to restart your shell (and Jupyter) for this to be usable here.

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Auxiliary Functions

Though the OpenAI library itself does most of the heavy lifting, it is still useful to have some auxiliary functions to help with our particular use.

In [4]:
def gpt3_complete(prompt, engine='text-davinci-003', temperature=0.1, max_tokens=200,
                  top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None,
                  echo=False):

    #print(f"DEBUG: {prompt}")
    response = openai.Completion.create(
      engine=engine,
      prompt=prompt,
      temperature=temperature,
      max_tokens=max_tokens,
      top_p=top_p,
      frequency_penalty=frequency_penalty,
      presence_penalty=presence_penalty,
      stop=stop,
      echo=echo
    )

    #print(f"DEBUG: {response}")
    completion = response['choices'][0]['text']

    return completion

In [5]:
def apply_to_examples(examples, prompt_func, temperature=0.1):
    results = []
    for example in examples:
        print(f">>> INPUT: {example}")
        result = gpt3_complete(prompt_func(example), temperature=temperature)
        print(f">>> OUTPUT:\n{result}")
        results.append(result)
        print("========================================\n\n")
    return results

In [6]:
def extract_lines_from_result(result):
    """
    Extracts the lines from the result string.
    """
    lines = [line.strip(' -*') for line in result.split('\n') if len(line) > 0]
    return lines

In [7]:
def string_to_tuples(s):
    """"
    Converts a string that looks like a tuple to an actual Python tuple.
    """
    return [eval(s.strip()) for s in extract_lines_from_result(s)]

In [8]:
def extract_terms_from_all_results(results):
    """
    Extracts the terms from the result string.
    """
    terms = []
    for result in results:
        terms.append(extract_lines_from_result(result))
    return terms

## Example Data
Let's conveniently have some standard data to exercise our solution below. That shall include *information input* to the system, as well as related later *queries*.

In [9]:
example_information_to_save = ["Flu shot cost = $80", 
                               "things my boss likes: cricket, science and vegetarian food",
                               "my wife wants a vegetarian food book",
                               "sales guy email = jp@example.com",
                               "vanessa's email vanessa@outlook.com, rember to send the ppts she asked",
                               "+55 11 27670-0987 -> pedro whatsapp",
                               "need to buy milk, eggs and bread", 
                               "need to sell old video game, chair",
                               "december receipts for gym: yoga, ballet, ??",
                               "book with KWG the hardware setup",
                               "dog day with the foreign visitors",
                               "event support: we failed :-(",
                               "floor layout: we forgot various details!!",
                               "first aid kit in the reception",
                               "ask the pediatrician about when to start brushing teeth"]

In [10]:
example_queries = ["What is the cost of a flu shot?", 
                   "shopping list", 
                   "what does my boss like?", 
                   "books my wife wants", 
                   "vegetarian food",
                   "emails",
                   "questions for the pediatrician"]

## Prompt Exercises: Inputing Information

### Prompt 1

A very simple prompt.

In [10]:
def extraction_prompt_1(x):

    prompt =\
f"""
Extract pieces of information, like phone numbers, email addresses, names, trivia, reminders, etc.
Input: {x}
"""
    return prompt 

In [11]:
apply_to_examples(example_information_to_save, extraction_prompt_1, temperature=0.5)

>>> INPUT: Flu shot cost = $80
>>> OUTPUT:

Output: $80


>>> INPUT: things my boss likes: cricket, science and vegetarian food
>>> OUTPUT:

Output: Cricket, Science, Vegetarian Food


>>> INPUT: my wife wants a vegetarian food book
>>> OUTPUT:

Output: None


>>> INPUT: sales guy email = jp@example.com
>>> OUTPUT:

Output: jp@example.com


>>> INPUT: vanessa's email vanessa@outlook.com, rember to send the ppts she asked
>>> OUTPUT:

Output:
Name: Vanessa 
Email: vanessa@outlook.com
Reminder: Send the PPTs


>>> INPUT: +55 11 27670-0987 -> pedro whatsapp
>>> OUTPUT:

Output: Phone number: +55 11 27670-0987, Name: Pedro


>>> INPUT: need to buy milk, eggs and bread
>>> OUTPUT:

Output: None


>>> INPUT: need to sell old video game, chair
>>> OUTPUT:

Output: None


>>> INPUT: december receipts for gym: yoga, ballet, ??
>>> OUTPUT:

Output: None


>>> INPUT: book with KWG the hardware setup
>>> OUTPUT:

Output: KWG (hardware setup)


>>> INPUT: dog day with the foreign visitors
>>> OUTPU

['\nOutput: $80',
 '\nOutput: Cricket, Science, Vegetarian Food',
 '\nOutput: None',
 '\nOutput: jp@example.com',
 '\nOutput:\nName: Vanessa \nEmail: vanessa@outlook.com\nReminder: Send the PPTs',
 '\nOutput: Phone number: +55 11 27670-0987, Name: Pedro',
 '\nOutput: None',
 '\nOutput: None',
 '\nOutput: None',
 '\nOutput: KWG (hardware setup)',
 '\nOutput: None',
 '\nOutput: No information extracted.',
 '\nOutput:\nReminder: Floor layout details',
 '\nOutput: No information extracted.',
 '\nOutput: None']

This is not particularly useful. For one example, the output is just a rewording of the input, and for another example there's no meaningful output at all. Let's try to be more specific on the kinds of outputs we want.

### Prompt 2

Now including some desired output structure and semantics.

In [12]:
def extraction_prompt_2(x):

    prompt =\
f"""
Extract pieces of personal information, like phone numbers, email addresses, names, trivia, reminders, etc., as tuples with the following format: (Category, Key, Value)
Input: {x}
"""
    return prompt 

In [13]:
apply_to_examples(example_information_to_save, extraction_prompt_2, temperature=0.5)

>>> INPUT: Flu shot cost = $80
>>> OUTPUT:

Output: ('Health', 'Flu Shot Cost', '$80')


>>> INPUT: things my boss likes: cricket, science and vegetarian food
>>> OUTPUT:

Output: (Interests, Cricket, True), (Interests, Science, True), (Interests, Vegetarian Food, True)


>>> INPUT: my wife wants a vegetarian food book
>>> OUTPUT:

Output: (Category: Reminder, Key: Wife, Value: Vegetarian Food Book)


>>> INPUT: sales guy email = jp@example.com
>>> OUTPUT:

Output: ("Personal Information", "Sales Guy Email", "jp@example.com")


>>> INPUT: vanessa's email vanessa@outlook.com, rember to send the ppts she asked
>>> OUTPUT:

Output: ("email", "vanessa", "vanessa@outlook.com"), ("reminder", "send ppts", "she asked")


>>> INPUT: +55 11 27670-0987 -> pedro whatsapp
>>> OUTPUT:

Output: ('Phone Number', 'Pedro Whatsapp', '+55 11 27670-0987')


>>> INPUT: need to buy milk, eggs and bread
>>> OUTPUT:

Output: (Reminder, Shopping List, Milk, Eggs, Bread)


>>> INPUT: need to sell old video game,

["\nOutput: ('Health', 'Flu Shot Cost', '$80')",
 '\nOutput: (Interests, Cricket, True), (Interests, Science, True), (Interests, Vegetarian Food, True)',
 '\nOutput: (Category: Reminder, Key: Wife, Value: Vegetarian Food Book)',
 '\nOutput: ("Personal Information", "Sales Guy Email", "jp@example.com")',
 '\nOutput: ("email", "vanessa", "vanessa@outlook.com"), ("reminder", "send ppts", "she asked")',
 "\nOutput: ('Phone Number', 'Pedro Whatsapp', '+55 11 27670-0987')",
 '\nOutput: (Reminder, Shopping List, Milk, Eggs, Bread)',
 '\nOutput: (Category, Key, Value): ("Item", "Video Game", "Sell"), ("Item", "Chair", "Sell")',
 '\nOutput: ("Gym", "Yoga", "December Receipts"), ("Gym", "Ballet", "December Receipts")',
 "\nOutput: ('Book', 'KWG', 'The Hardware Setup')",
 'Output: No personal information found.',
 '\nOutput: (Event, Support, Failed)',
 '\nOutput: (Category, Key, Value): (Floor Layout, Details, Various)',
 '\nOutput: (Reminder, "First Aid Kit", "Reception")',
 '\nOutput: (Category

We see various problems: the result includes the word "Output", but we really only want the tuples; multiple facts sometimes are separated by commas, instead of newlines, as we'd prefer; some extractions are just wrong. Adding some examples might make our intent more clear. So let's try another prompt.

### Prompt 3

Now including an example.

In [14]:
def extraction_prompt_3(x):

    prompt =\
f"""
Extract pieces of personal information, like phone numbers, email addresses, names, trivia, reminders, etc., as tuples with the following format: (Category, Key, Value)

Example input: "Mom's phone number is 555-555-5555"
Example output: ("Family", "mom's phone number", "555-555-5555")

Input: {x}
Output: 
"""
    return prompt 

In [15]:
apply_to_examples(example_information_to_save, extraction_prompt_3, temperature=0.5)

>>> INPUT: Flu shot cost = $80
>>> OUTPUT:
("Health", "Flu shot cost", "$80")


>>> INPUT: things my boss likes: cricket, science and vegetarian food
>>> OUTPUT:
("Interests", "boss likes cricket", "true")
("Interests", "boss likes science", "true")
("Interests", "boss likes vegetarian food", "true")


>>> INPUT: my wife wants a vegetarian food book
>>> OUTPUT:
("Personal", "wife's preference", "vegetarian food book")


>>> INPUT: sales guy email = jp@example.com
>>> OUTPUT:
("Work", "sales guy email", "jp@example.com")


>>> INPUT: vanessa's email vanessa@outlook.com, rember to send the ppts she asked
>>> OUTPUT:
("Personal", "Vanessa's email", "vanessa@outlook.com"),
("Reminder", "Send PPTs", "Vanessa asked")


>>> INPUT: +55 11 27670-0987 -> pedro whatsapp
>>> OUTPUT:
("Contacts", "Pedro WhatsApp", "+55 11 27670-0987")


>>> INPUT: need to buy milk, eggs and bread
>>> OUTPUT:
("Shopping List", "milk", "buy")
("Shopping List", "eggs", "buy")
("Shopping List", "bread", "buy")


>>> IN

['("Health", "Flu shot cost", "$80")',
 '("Interests", "boss likes cricket", "true")\n("Interests", "boss likes science", "true")\n("Interests", "boss likes vegetarian food", "true")',
 '("Personal", "wife\'s preference", "vegetarian food book")',
 '("Work", "sales guy email", "jp@example.com")',
 '("Personal", "Vanessa\'s email", "vanessa@outlook.com"),\n("Reminder", "Send PPTs", "Vanessa asked")',
 '("Contacts", "Pedro WhatsApp", "+55 11 27670-0987")',
 '("Shopping List", "milk", "buy")\n("Shopping List", "eggs", "buy")\n("Shopping List", "bread", "buy")',
 '("Item", "video game", "sell")\n("Item", "chair", "sell")',
 '("Finance", "December Receipts", "Gym: Yoga, Ballet, ??")',
 '("Hardware", "KWG setup", "book")',
 '("Activities", "dog day", "with the foreign visitors")',
 '("Event Support", "Status", "Failed")',
 '("Layout", "floor", "various details")',
 '("Miscellaneous", "first aid kit", "reception")',
 '("Health", "pediatrician brushing teeth advice", "when to start brushing te

We can now see a number of improvements: fact strings are consistently quoted; multiple facts are put one per line; the information recorded is overall more detailed; the “Sales” category sounds more appropriate than the more general "Email" we had before. There are, however, still some oddities. In the second example, the boss’ preferences are awkwardly given as a Boolean list; in the last example, it would be better to break “yoga, ballet, ??”  in separate facts; and the categories sound all rather random. 

### Prompt 4

To address the above points, in the next prompt we introduce a multi-fact example, an additional assumption regarding multiple facts, and a constraint on the valid categories. This is now a rather dynamic prompt: both the user input and the valid categories are parameters. In general, we can use dynamic prompts to make them more general and connect better to the client application.

In [16]:
def extraction_prompt_4(x, categories=["Family", "Work", "Friends", "Shopping", 
                                       "Health", "Finance", "Travel", "Home", 
                                       "Pets", "Hobbies", "Other"]):

    prompt =\
f"""
Extract pieces of personal information, like phone numbers, email addresses, names, trivia, reminders, etc., as tuples with the following format: (Category, Key, Value)
Assume everything mentioned refers to the same thing. Constraints:
  - Allowed Categories: {', '.join(categories)}


Example input: "Mom's phone number is 555-555-5555"
Example output: ("Family", "mom's phone number", "555-555-5555")

Example input: "Need to do: lab work, ultrasound, buy aspirin"
Example output: 
("Health", "to do", "lab work")
("Health", "to do", "ultrasound")
("Health", "buy", "aspirin")	

Input: {x}
Output: 
"""
    return prompt 

In [17]:
apply_to_examples(example_information_to_save, extraction_prompt_4, temperature=0.5)

>>> INPUT: Flu shot cost = $80
>>> OUTPUT:
("Health", "flu shot cost", "$80")


>>> INPUT: things my boss likes: cricket, science and vegetarian food
>>> OUTPUT:
("Work", "likes", "cricket")
("Work", "likes", "science")
("Work", "likes", "vegetarian food")


>>> INPUT: my wife wants a vegetarian food book
>>> OUTPUT:
("Shopping", "vegetarian food book", "wife")


>>> INPUT: sales guy email = jp@example.com
>>> OUTPUT:
("Work", "sales guy email", "jp@example.com")


>>> INPUT: vanessa's email vanessa@outlook.com, rember to send the ppts she asked
>>> OUTPUT:
("Work", "vanessa's email", "vanessa@outlook.com")
("Work", "to do", "send the ppts")


>>> INPUT: +55 11 27670-0987 -> pedro whatsapp
>>> OUTPUT:
("Friends", "pedro whatsapp", "+55 11 27670-0987")


>>> INPUT: need to buy milk, eggs and bread
>>> OUTPUT:
("Shopping", "buy", "milk")
("Shopping", "buy", "eggs")
("Shopping", "buy", "bread")


>>> INPUT: need to sell old video game, chair
>>> OUTPUT:
("Shopping", "sell", "old video gam

['("Health", "flu shot cost", "$80")',
 '("Work", "likes", "cricket")\n("Work", "likes", "science")\n("Work", "likes", "vegetarian food")',
 '("Shopping", "vegetarian food book", "wife")',
 '("Work", "sales guy email", "jp@example.com")',
 '("Work", "vanessa\'s email", "vanessa@outlook.com")\n("Work", "to do", "send the ppts")',
 '("Friends", "pedro whatsapp", "+55 11 27670-0987")',
 '("Shopping", "buy", "milk")\n("Shopping", "buy", "eggs")\n("Shopping", "buy", "bread")',
 '("Shopping", "sell", "old video game")\n("Shopping", "sell", "chair")',
 '("Hobbies", "gym", "yoga")\n("Hobbies", "gym", "ballet")',
 '("Hobbies", "book", "KWG the hardware setup")',
 '("Other", "dog day", "with the foreign visitors")',
 '("Other", "event support", "we failed")',
 '("Home", "floor layout", "we forgot various details!!")',
 '("Home", "first aid kit", "reception")',
 '("Health", "pediatrician question", "when to start brushing teeth")']

This is better: the categories comply with our constraints and inputs are broken in various facts whena pproperiate. However, we note that the category “Hobbies” is misleading, particularly because nothing is said of the receipts! The pediatrician one is also terser than we wanted now. 

### Prompt 5

Let'us increase the expressivity of our facts. By adding new fields *Type* and *People* we hope to be able to absorb the information provided more effectivelly.

In [18]:
def extraction_prompt_5(x, categories=["Family", "Work", "Friends", "Shopping", 
                                       "Health", "Finance", "Travel", "Home", 
                                       "Pets", "Hobbies", "Other"]):

    prompt =\
f"""
Extract pieces of personal information, like phone numbers, email addresses, names, trivia, reminders, etc., as tuples with the following format: (Category, Type, People, Key, Value)
Assume everything mentioned refers to the same thing. Constraints:
  - Allowed Categories: {', '.join(categories)}
  - Allowed Types: "List", "Email", "Phone", "Address", "Document", "Pendency", "Price", "Reminder", "Note", "Doubt", "Wish", "Other"
  - People contain the name or description of the people or organizations concerned, or is empty if no person or organization is mentioned.
  
Example input: "Mom's phone number is 555-555-5555"
Example output: ("Family", "Phone", "mom", "mom's number", "555-555-5555")

Example input: "email of the building administration = adm@example.com"
Example output: ("Work", "Email", "building administration", "email", "adm@example.com")

Example input: "Need to do: lab work, ultrasound, buy aspirin"
Example output: 
("Health", "List", "", "to do", "lab work")
("Health", "List", "", "to do", "ultrasound")
("Shopping", "List", "", "aspirin", "buy")	

Input: {x}
Output: 
"""
    return prompt 

In [19]:
apply_to_examples(example_information_to_save, extraction_prompt_5, temperature=0.5)

>>> INPUT: Flu shot cost = $80
>>> OUTPUT:
("Health", "Price", "", "flu shot cost", "$80")


>>> INPUT: things my boss likes: cricket, science and vegetarian food
>>> OUTPUT:
("Work", "List", "boss", "likes", "cricket")
("Work", "List", "boss", "likes", "science")
("Work", "List", "boss", "likes", "vegetarian food")


>>> INPUT: my wife wants a vegetarian food book
>>> OUTPUT:
("Home", "Wish", "wife", "vegetarian food book", "")


>>> INPUT: sales guy email = jp@example.com
>>> OUTPUT:
("Work", "Email", "sales guy", "email", "jp@example.com")


>>> INPUT: vanessa's email vanessa@outlook.com, rember to send the ppts she asked
>>> OUTPUT:
("Friends", "Email", "Vanessa", "Vanessa's email", "vanessa@outlook.com")
("Friends", "Reminder", "Vanessa", "send ppts", "send")


>>> INPUT: +55 11 27670-0987 -> pedro whatsapp
>>> OUTPUT:
("Friends", "Phone", "Pedro", "WhatsApp", "+55 11 27670-0987")


>>> INPUT: need to buy milk, eggs and bread
>>> OUTPUT:
("Shopping", "List", "", "milk", "buy")
("S

['("Health", "Price", "", "flu shot cost", "$80")',
 '("Work", "List", "boss", "likes", "cricket")\n("Work", "List", "boss", "likes", "science")\n("Work", "List", "boss", "likes", "vegetarian food")',
 '("Home", "Wish", "wife", "vegetarian food book", "")',
 '("Work", "Email", "sales guy", "email", "jp@example.com")',
 '("Friends", "Email", "Vanessa", "Vanessa\'s email", "vanessa@outlook.com")\n("Friends", "Reminder", "Vanessa", "send ppts", "send")',
 '("Friends", "Phone", "Pedro", "WhatsApp", "+55 11 27670-0987")',
 '("Shopping", "List", "", "milk", "buy")\n("Shopping", "List", "", "eggs", "buy")\n("Shopping", "List", "", "bread", "buy")',
 '("Home", "List", "", "to sell", "old video game")\n("Home", "List", "", "to sell", "chair")',
 '("Finance", "Document", "gym", "receipts", "december")\n("Finance", "List", "gym", "yoga", "")\n("Finance", "List", "gym", "ballet", "")\n("Finance", "Doubt", "gym", "??", "")',
 '("Other", "Document", "KWG", "hardware setup", "book")',
 '("Pets", "Rem

This is very good already! We do note, however, that the "december receipts" results is strangely messed up. To fix that, instead of changing the prompt, we'll try to reduce the *temperature* parameter given to GPT-3, which controls how "creative" the output gets.

In [20]:
apply_to_examples(example_information_to_save, extraction_prompt_5, temperature=0.1)

>>> INPUT: Flu shot cost = $80
>>> OUTPUT:
("Health", "Price", "", "flu shot", "$80")


>>> INPUT: things my boss likes: cricket, science and vegetarian food
>>> OUTPUT:
("Work", "List", "boss", "likes", "cricket")
("Work", "List", "boss", "likes", "science")
("Work", "List", "boss", "likes", "vegetarian food")


>>> INPUT: my wife wants a vegetarian food book
>>> OUTPUT:
("Home", "Wish", "wife", "vegetarian food book", "")


>>> INPUT: sales guy email = jp@example.com
>>> OUTPUT:
("Work", "Email", "sales guy", "email", "jp@example.com")


>>> INPUT: vanessa's email vanessa@outlook.com, rember to send the ppts she asked
>>> OUTPUT:
("Friends", "Email", "vanessa", "vanessa's email", "vanessa@outlook.com")
("Friends", "Reminder", "vanessa", "send ppts", "")


>>> INPUT: +55 11 27670-0987 -> pedro whatsapp
>>> OUTPUT:
("Friends", "Phone", "Pedro", "Pedro's WhatsApp", "+55 11 27670-0987")


>>> INPUT: need to buy milk, eggs and bread
>>> OUTPUT:
("Shopping", "List", "", "milk", "buy")
("Sh

['("Health", "Price", "", "flu shot", "$80")',
 '("Work", "List", "boss", "likes", "cricket")\n("Work", "List", "boss", "likes", "science")\n("Work", "List", "boss", "likes", "vegetarian food")',
 '("Home", "Wish", "wife", "vegetarian food book", "")',
 '("Work", "Email", "sales guy", "email", "jp@example.com")',
 '("Friends", "Email", "vanessa", "vanessa\'s email", "vanessa@outlook.com")\n("Friends", "Reminder", "vanessa", "send ppts", "")',
 '("Friends", "Phone", "Pedro", "Pedro\'s WhatsApp", "+55 11 27670-0987")',
 '("Shopping", "List", "", "milk", "buy")\n("Shopping", "List", "", "eggs", "buy")\n("Shopping", "List", "", "bread", "buy")',
 '("Home", "List", "", "sell", "video game")\n("Home", "List", "", "sell", "chair")',
 '("Finance", "Document", "gym", "december receipts", "yoga")\n("Finance", "Document", "gym", "december receipts", "ballet")\n("Finance", "Document", "gym", "december receipts", "??")',
 '("Other", "Document", "KWG", "hardware setup", "book")',
 '("Pets", "Reminde

That looks very good now! We'll thus stop iterating on this prompt. To finalize, let's just rename the best prompt function to denote its importance.

In [21]:
best_input_prompt = extraction_prompt_5

## Prompt Exercises: Querying Information

We have just gone through some iterations of the information input prompt and we are satisfied with results for the time being. Now we can give the next step: populate a database using this mechanism, and then engineering a prompt to be able to query this database! Let's get the input tuples.

In [22]:
input_tuples = []
for input in example_information_to_save:
    for tuples in string_to_tuples(gpt3_complete(best_input_prompt(input))):
        input_tuples.append(tuples)
        
input_tuples

[('Health', 'Price', '', 'flu shot', '$80'),
 ('Work', 'List', 'boss', 'likes', 'cricket'),
 ('Work', 'List', 'boss', 'likes', 'science'),
 ('Work', 'List', 'boss', 'likes', 'vegetarian food'),
 ('Home', 'Wish', 'wife', 'vegetarian food book', ''),
 ('Work', 'Email', 'sales guy', 'email', 'jp@example.com'),
 ('Friends', 'Email', 'vanessa', "vanessa's email", 'vanessa@outlook.com'),
 ('Friends', 'Reminder', 'vanessa', 'send ppts', ''),
 ('Friends', 'Phone', 'Pedro', "Pedro's WhatsApp", '+55 11 27670-0987'),
 ('Shopping', 'List', '', 'milk', 'buy'),
 ('Shopping', 'List', '', 'eggs', 'buy'),
 ('Shopping', 'List', '', 'bread', 'buy'),
 ('Home', 'List', '', 'sell', 'video game'),
 ('Home', 'List', '', 'sell', 'chair'),
 ('Finance', 'Document', 'gym', 'december receipts', 'yoga'),
 ('Finance', 'Document', 'gym', 'december receipts', 'ballet'),
 ('Finance', 'Document', 'gym', 'december receipts', '??'),
 ('Other', 'Document', 'KWG', 'hardware setup', 'book'),
 ('Pets', 'Reminder', 'foreign vi

For simplicity, in this exercise our "database" shall be just a Pandas dataframe, but naturally this can be extended to any actual database system. Below we populate it with the information we have already parsed.

In [23]:
database = pd.DataFrame(input_tuples, columns=["Category", "Type", "People", "Key", "Value"])
database

,Category,Type,People,Key,Value
0,Health,Price,,flu shot,$80
1,Work,List,boss,likes,cricket
2,Work,List,boss,likes,science
3,Work,List,boss,likes,vegetarian food
4,Home,Wish,wife,vegetarian food book,
5,Work,Email,sales guy,email,jp@example.com
6,Friends,Email,vanessa,vanessa's email,vanessa@outlook.com
7,Friends,Reminder,vanessa,send ppts,
8,Friends,Phone,Pedro,Pedro's WhatsApp,+55 11 27670-0987
9,Shopping,List,,milk,buy


Now we can begin experimenting with the prompts for querying.

### Prompt 1 (querying):

A very naive solution that must inspect each row of the dataframe. Might work, but it seems too costly.

In [24]:
def querying_promt_1(query, example):

    prompt = \
f"""
Determine whether the query "{query}" is related to the tuple "{example}".
Answer (yes/no):
"""

    return prompt

In [25]:
for query in example_queries:
    print(f"INPUT QUERY: {query}")
    for i, row in database.iterrows():
        row_string = f"{tuple(row.values)}"
        print(f"INPUT ROW: {row_string}")
        print(f"OUTPUT: {gpt3_complete(querying_promt_1(query, row_string))}")
    
    print(f"====================")

INPUT QUERY: What is the cost of a flu shot?
INPUT ROW: ('Health', 'Price', '', 'flu shot', '$80')
OUTPUT: Yes
INPUT ROW: ('Work', 'List', 'boss', 'likes', 'cricket')
OUTPUT: No
INPUT ROW: ('Work', 'List', 'boss', 'likes', 'science')
OUTPUT: No
INPUT ROW: ('Work', 'List', 'boss', 'likes', 'vegetarian food')
OUTPUT: No
INPUT ROW: ('Home', 'Wish', 'wife', 'vegetarian food book', '')
OUTPUT: No
INPUT ROW: ('Work', 'Email', 'sales guy', 'email', 'jp@example.com')
OUTPUT: No
INPUT ROW: ('Friends', 'Email', 'vanessa', "vanessa's email", 'vanessa@outlook.com')
OUTPUT: No
INPUT ROW: ('Friends', 'Reminder', 'vanessa', 'send ppts', '')
OUTPUT: No
INPUT ROW: ('Friends', 'Phone', 'Pedro', "Pedro's WhatsApp", '+55 11 27670-0987')
OUTPUT: No
INPUT ROW: ('Shopping', 'List', '', 'milk', 'buy')
OUTPUT: No
INPUT ROW: ('Shopping', 'List', '', 'eggs', 'buy')
OUTPUT: No
INPUT ROW: ('Shopping', 'List', '', 'bread', 'buy')
OUTPUT: No
INPUT ROW: ('Home', 'List', '', 'sell', 'video game')
OUTPUT: No
INPUT ROW:

### Prompt 2
What if instead we extract some key terms from the query the result for searching?

In [13]:
def querying_promt_2(query):

    prompt = \
f"""
Extract the main entities (one per line, without bullets) in the following sentence: "{query}"
"""

    return prompt

In [14]:
results = apply_to_examples(example_queries, querying_promt_2, temperature=0.5)

>>> INPUT: What is the cost of a flu shot?
>>> OUTPUT:

Flu shot 
Cost


>>> INPUT: shopping list
>>> OUTPUT:

Shopping 
List


>>> INPUT: what does my boss like?
>>> OUTPUT:

Boss


>>> INPUT: books my wife wants
>>> OUTPUT:

Books
Wife


>>> INPUT: vegetarian food
>>> OUTPUT:

Vegetarian 
Food


>>> INPUT: emails
>>> OUTPUT:

Emails


>>> INPUT: questions for the pediatrician
>>> OUTPUT:

Pediatrician
Questions




In [15]:
examples_terms = extract_terms_from_all_results(results)
examples_terms

[['Flu shot ', 'Cost'],
 ['Shopping ', 'List'],
 ['Boss'],
 ['Books', 'Wife'],
 ['Vegetarian ', 'Food'],
 ['Emails'],
 ['Pediatrician', 'Questions']]

In [16]:
best_terms_extraction_prompt = querying_promt_2

### Prompt 3

As we can see, the entities extraction seem to work. But what if we need a synonym? Let's try to perform some data augmentation to complement the previous prompt.

In [17]:
def querying_promt_3(term):

    prompt = \
f"""
List some synonyms to the following term: "{term}"
Synonyms (one synonym per line):
"""

    return prompt

In [18]:
augmentation_raw_results = []
for terms in examples_terms:
    augmentation_raw_result = apply_to_examples(terms, querying_promt_3)
    augmentation_raw_results.append(augmentation_raw_result)

augmentation_raw_results

>>> INPUT: Flu shot 
>>> OUTPUT:

Vaccination
Injection
Immunization
Jab


>>> INPUT: Cost
>>> OUTPUT:

Expense
Price
Charge
Fee
Tariff
Outlay


>>> INPUT: Shopping 
>>> OUTPUT:

Purchasing
Acquiring
Buying
Procuring
Obtaining


>>> INPUT: List
>>> OUTPUT:

Inventory
Catalogue
Register
Index
Roll
Table


>>> INPUT: Boss
>>> OUTPUT:

Supervisor
Manager
Chief
Leader
Director
Head
Governor
Foreman
Principal
Executive


>>> INPUT: Books
>>> OUTPUT:

Publications
Texts
Manuscripts
Scripts
Tomes
Volumes


>>> INPUT: Wife
>>> OUTPUT:

Spouse
Partner
Helpmeet
Consort
Matron


>>> INPUT: Vegetarian 
>>> OUTPUT:

Herbivore
Fruitarian
Lacto-ovo vegetarian
Pescatarian
Pollotarian


>>> INPUT: Food
>>> OUTPUT:

Nourishment
Sustenance
Nutriment
Provisions
Victuals
Fare


>>> INPUT: Emails
>>> OUTPUT:

Messages
Correspondence
Communications
Letters
Memos


>>> INPUT: Pediatrician
>>> OUTPUT:

Child Doctor
Paediatrician
Infant Specialist
Kids Physician
Baby Physician


>>> INPUT: Questions
>>> OUTPUT:

[['\nVaccination\nInjection\nImmunization\nJab',
  '\nExpense\nPrice\nCharge\nFee\nTariff\nOutlay'],
 ['\nPurchasing\nAcquiring\nBuying\nProcuring\nObtaining',
  '\nInventory\nCatalogue\nRegister\nIndex\nRoll\nTable'],
 ['\nSupervisor\nManager\nChief\nLeader\nDirector\nHead\nGovernor\nForeman\nPrincipal\nExecutive'],
 ['\nPublications\nTexts\nManuscripts\nScripts\nTomes\nVolumes',
  '\nSpouse\nPartner\nHelpmeet\nConsort\nMatron'],
 ['\nHerbivore\nFruitarian\nLacto-ovo vegetarian\nPescatarian\nPollotarian',
  '\nNourishment\nSustenance\nNutriment\nProvisions\nVictuals\nFare'],
 ['\nMessages\nCorrespondence\nCommunications\nLetters\nMemos'],
 ['\nChild Doctor\nPaediatrician\nInfant Specialist\nKids Physician\nBaby Physician',
  '\nInquiries\nQueries\nInterrogations\nEnquiries\nQuizzes\nExaminations']]

In [32]:
example_augmentations = []
for raw_result in augmentation_raw_results:
    example_augmentation = extract_terms_from_all_results(raw_result)
    flat_example_augmentation = [item for sublist in example_augmentation for item in sublist]
    example_augmentations.append(flat_example_augmentation)

example_augmentations    

[['Vaccination',
  'Injection',
  'Immunization',
  'Expense',
  'Price',
  'Fee',
  'Charge',
  'Tariff',
  'Outlay'],
 ['Purchasing',
  'Acquiring',
  'Buying',
  'Obtaining',
  'Procuring',
  'Inventory',
  'Catalogue',
  'Register',
  'Index',
  'Roll',
  'Table'],
 ['Supervisor',
  'Manager',
  'Chief',
  'Leader',
  'Head',
  'Director',
  'Foreman',
  'Governor',
  'Principal'],
 ['Publications',
  'Texts',
  'Manuscripts',
  'Tomes',
  'Volumes',
  'Spouse',
  'Partner',
  'Helpmeet',
  'Consort',
  'Matron'],
 ['Herbivore',
  'Fruitarian',
  'Lacto-ovo vegetarian',
  'Plant-based',
  'Nourishment',
  'Sustenance',
  'Nutriment',
  'Provisions',
  'Victuals',
  'Fare'],
 ['Message', 'Correspondence', 'Communication', 'Dispatch', 'Memo', 'Post'],
 ['Child Doctor',
  'Paediatrician',
  'Infant Specialist',
  'Kids Physician',
  'Inquiries',
  'Queries',
  'Interrogations',
  'Enquiries',
  'Quizzes']]

In [33]:
best_augmentation_prompt = querying_promt_3

### Querying Demo

We can now actually query our database! Naturally, a proper search mechanism would be much more sophisticated, but this does show the potential of the approach. Below we decouple a basic filtering mechanism from the actual keyword search.

In [34]:
def database_filtered_by(df, categories=None, entry_types=None, people=None):

    def aux_filter(df, column, values):
        if values is not None and len(values) > 0:
            return df[df[column].str.lower().isin([v.lower() for v in values])]
        else:
            return df
    
    df = aux_filter(df, "Category", categories)
    df = aux_filter(df, "Type", entry_types)
    df = aux_filter(df, "People", people)
        
    return df

In [35]:
def search_dataframe(df, original_terms, augmented_terms):
    """
    Searches the database for the specified terms.
    """
    all_terms = original_terms + augmented_terms
    df = df.fillna("")

    df_results = None
    for column in df.columns:
        df_result = df[df[column].str.contains("|".join(all_terms), case=False).fillna(False)]
        if df_results is None:
            df_results = df_result
        else:
            df_results = pd.concat([df_results, df_result])
            
    return df_results

In [36]:
database

,Category,Type,People,Key,Value
0,Health,Price,,flu shot,$80
1,Work,List,boss,likes,cricket
2,Work,List,boss,likes,science
3,Work,List,boss,likes,vegetarian food
4,Home,Wish,wife,vegetarian food book,
5,Work,Email,sales guy,email,jp@example.com
6,Friends,Email,vanessa,vanessa's email,vanessa@outlook.com
7,Friends,Reminder,vanessa,send ppts,
8,Friends,Phone,Pedro,Pedro's WhatsApp,+55 11 27670-0987
9,Shopping,List,,milk,buy


In [37]:
for i, original_terms in enumerate(examples_terms):
    augmented_terms = example_augmentations[i]
    print(f"Search terms: {original_terms}")
    print(f"Augmented terms: {augmented_terms}")
    print(search_dataframe(database, original_terms, augmented_terms))
    print(f"====================")

Search terms: ['Flu shot', 'Cost']
Augmented terms: ['Vaccination', 'Injection', 'Immunization', 'Expense', 'Price', 'Fee', 'Charge', 'Tariff', 'Outlay']
  Category   Type People       Key Value
0   Health  Price         flu shot   $80
0   Health  Price         flu shot   $80
Search terms: ['Shopping ', 'List']
Augmented terms: ['Purchasing', 'Acquiring', 'Buying', 'Obtaining', 'Procuring', 'Inventory', 'Catalogue', 'Register', 'Index', 'Roll', 'Table']
    Category  Type People            Key            Value
1       Work  List   boss          likes          cricket
2       Work  List   boss          likes          science
3       Work  List   boss          likes  vegetarian food
9   Shopping  List                  milk              buy
10  Shopping  List                  eggs              buy
11  Shopping  List                 bread              buy
12      Home  List                  sell       video game
13      Home  List                  sell            chair
21      Home  List  

## Complete Solution

Now that we have seen the individual components of the solution, we can put them together into two functions: one for inputing information, and one for querying. These are essentially what we'll be using in the final application, transporting from our notebook studies to the actual product.

In [38]:
facts_database = pd.DataFrame([], columns=["Category", "Type", "People", "Key", "Value"])

In [39]:
def extract_facts(facts_utterance):
    fact_tuples = string_to_tuples(gpt3_complete(best_input_prompt(facts_utterance)))
    return fact_tuples

In [40]:
def insert_facts(facts_utterance, database):
    """
    Inserts a fact into the database.
    """
    fact_tuples = extract_facts(facts_utterance)
    print(f"Facts: {fact_tuples}")
    for fact_tuple in fact_tuples:
        # we add the tuple only if at least one of the important information fields is not empty
        if len(fact_tuple[2]) > 0 or len(fact_tuple[3]) > 0 or len(fact_tuple[4]) > 0:
            df_to_add = pd.DataFrame([fact_tuple], columns=["Category", "Type", "People", "Key", "Value"])
            database = pd.concat([database, df_to_add], ignore_index=True)
    return database

In [41]:
def query(fact_query, database, categories=None, entry_types=None, people=None, verbose=False):
    """
    Queries the database for a fact. If requested, prior to keyword search, filters the database 
    by categories, entry types and people.
    """
    raw_original_terms = gpt3_complete(best_terms_extraction_prompt(fact_query))
    original_terms = extract_lines_from_result(raw_original_terms)
    if verbose:
        print(original_terms)

    augmented_terms = []
    for original_term in original_terms:
        raw_augmented_terms = gpt3_complete(best_augmentation_prompt(original_term))
        augmented_terms += extract_lines_from_result(raw_augmented_terms)
    if verbose:
        print(augmented_terms)
    
    
    return search_dataframe(database_filtered_by(database, categories, entry_types, people), 
                            original_terms, augmented_terms)


### Early Demo
Before you actually invest more time in building an application, it is perhaps wise to exercise it right here in a notebook. You can do it yourself, or you can invite some other stakeholders to do it. It is unlikely that your non-technical stakeholders will be able to use the notebook, so ideally you should try some scenarios together, with you operating the notebook and them providing the input and evaluation feedback.

In [42]:
facts_database

,Category,Type,People,Key,Value


In [43]:
facts_database = insert_facts("Flu shot cost = $80", database=facts_database)
facts_database = insert_facts("Don't forget car checkup", database=facts_database)
facts_database = insert_facts("Buy MSFT stock at $250", database=facts_database)
facts_database = insert_facts("MSFT stock PE ratio is around 25", database=facts_database)
facts_database = insert_facts("Mom's phone is 555-0000-1111", database=facts_database)

Facts: [('Health', 'Price', '', 'flu shot', '$80')]
Facts: [('Home', 'Reminder', '', 'car checkup', "don't forget")]
Facts: [('Finance', 'Price', 'MSFT', 'stock', '250'), ('Finance', 'Pendency', '', 'buy MSFT stock', '')]
Facts: [('Finance', 'Price', 'MSFT', 'PE ratio', '25')]
Facts: [('Family', 'Phone', 'mom', "mom's number", '555-0000-1111')]


In [44]:
facts_database = insert_facts("need to buy paper towels, tonic water and detergent", database=facts_database)

Facts: [('Shopping', 'List', '', 'paper towels', 'buy'), ('Shopping', 'List', '', 'tonic water', 'buy'), ('Shopping', 'List', '', 'detergent', 'buy')]


In [45]:
# try adding some nonsense
facts_database = insert_facts("ka lkaj kljakl jakl jla;a;;;;", database=facts_database)

Facts: [('Other', 'Other', '', '', '')]


In [46]:
facts_database = insert_facts("Colorless green ideas sleep furiously", database=facts_database)

Facts: [('Other', 'Note', '', 'ideas', 'colorless green'), ('Other', 'Note', '', 'ideas', 'sleep furiously')]


In [47]:
facts_database

,Category,Type,People,Key,Value
0,Health,Price,,flu shot,$80
1,Home,Reminder,,car checkup,don't forget
2,Finance,Price,MSFT,stock,250
3,Finance,Pendency,,buy MSFT stock,
4,Finance,Price,MSFT,PE ratio,25
5,Family,Phone,mom,mom's number,555-0000-1111
6,Shopping,List,,paper towels,buy
7,Shopping,List,,tonic water,buy
8,Shopping,List,,detergent,buy
9,Other,Note,,ideas,colorless green


In [48]:
query("vaccine cost", database=facts_database)

,Category,Type,People,Key,Value
0,Health,Price,,flu shot,$80
2,Finance,Price,MSFT,stock,250
4,Finance,Price,MSFT,PE ratio,25
0,Health,Price,,flu shot,$80


In [50]:
query("vaccine cost", database=facts_database, categories=["Health"])

,Category,Type,People,Key,Value
0,Health,Price,,flu shot,$80
0,Health,Price,,flu shot,$80


In [49]:
query("things to purchase", database=facts_database)

,Category,Type,People,Key,Value
3,Finance,Pendency,,buy MSFT stock,
6,Shopping,List,,paper towels,buy
7,Shopping,List,,tonic water,buy
8,Shopping,List,,detergent,buy
